## 使用一個神經網路解決 FizzBuzz 問題

輸入一個數字：
 - 可以被 15 整除就回傳 FizzBuzz；
 - 可以被 3 整除就回傳 Fizz；
 - 被 5 整除就回傳 Buzz；
 - 都不能整除就回傳原本的數字



## 用程式時做的話會長這樣

In [1]:
def fizz_buzz(num):
    if num % 15 == 0:
        return 'FizzBuzz'
    elif num % 3 == 0:
        return 'Fizz'
    elif num % 5 == 0:
        return 'Buzz'
    else:
        return str(num)
    
print(fizz_buzz(45))
print(fizz_buzz(48))
print(fizz_buzz(50))

FizzBuzz
Fizz
Buzz


## 其實就是一個分類問題，讓程式容易解讀

In [7]:
def fizz_buzz(num):
    if num % 15 == 0:
        return 0 # 'FizzBuzz'
    elif num % 3 == 0:
        return 1 # 'Fizz'
    elif num % 5 == 0:
        return 2 # 'Buzz'
    else:
        return 3 # ''

## 來看看深度學習怎麼解決這皮毛問題

In [8]:
import random
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

BATCH = 32
DIGITS = 10
EPOCH = 300
DATASET_SIZE = 3000
CLASSES = ['FizzBuzz', 'Fizz', 'Buzz', '']

In [19]:
!pip install torchsummary

fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
from torchsummary import summary

## 深度學習很難嗎？其實網路結構就這幾行

In [18]:
class FizzBuzz(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(FizzBuzz, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_channel, 1024), ## 全連接層
            nn.Dropout(0.5),             ## Dropout 避免 overfitting
            nn.ReLU(),                   ## Activation 函式
            nn.Linear(1024, 1024),       ## 全連接層
            nn.Dropout(0.5),             ## Dropout 避免 overfitting
            nn.ReLU(),                   ## Activation 函式
            nn.Linear(1024, out_channel) ## 全連接層，以此例來說 out_channel 應該為 4
        )

    def forward(self, x):
        x = self.layers(x)
        return x

## 可參考 lost 函式清單
 - https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [10]:
def training(model, optimizer, training_data):
    model.train()

    for data, label in training_data:
        data = Variable(torch.FloatTensor(data))
        label = Variable(torch.LongTensor(label))

        out = model(data)
        optimizer.zero_grad()
        classification_loss = F.cross_entropy(out, label) ## 常用分類問題的 loss 函式
        classification_loss.backward()
        optimizer.step()

In [21]:
m = FizzBuzz(DIGITS, 4)

In [22]:
summary(m, (13, 10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 13, 1024]          11,264
           Dropout-2             [-1, 13, 1024]               0
              ReLU-3             [-1, 13, 1024]               0
            Linear-4             [-1, 13, 1024]       1,049,600
           Dropout-5             [-1, 13, 1024]               0
              ReLU-6             [-1, 13, 1024]               0
            Linear-7                [-1, 13, 4]           4,100
Total params: 1,064,964
Trainable params: 1,064,964
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.61
Params size (MB): 4.06
Estimated Total Size (MB): 4.67
----------------------------------------------------------------


In [11]:
def testing(model, data):
    model.eval()
    loss = []
    correct = 0

    for x, y in data:
        x = Variable(torch.FloatTensor(x))
        y = Variable(torch.LongTensor(y))

        out = model(x)
        loss += [F.cross_entropy(out, y).data.item()]
        pred = out.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).sum()

    avg_loss = sum(loss) / len(loss)

    return {
        'accuracy': 100.0 * correct/(len(loss)*BATCH),
        'avg_loss': avg_loss
    }

In [12]:
def encoder(num):
    return list(map(lambda x: int(x), ('{:0' + str(DIGITS) + 'b}').format(num))) 

def make_data(num_of_data, batch_size):
    xs = []
    ys = []
    for _ in range(num_of_data):
        x = random.randint(0, 2**DIGITS-1)
        xs += [encoder(x)]
        ys += [fizz_buzz(x)]

    data = []
    for b in range(num_of_data//batch_size):
        xxs = xs[b*batch_size:(b+1)*batch_size]
        yys = ys[b*batch_size:(b+1)*batch_size]
        data += [(xxs, yys)]
        
    return data


In [24]:
training_data = make_data(DATASET_SIZE, BATCH)
testing_data = make_data(100, BATCH)

In [35]:
optimizer = optim.SGD(m.parameters(), lr=0.02, momentum=0.9)

print('==== 開始訓練 ====')
for epoch in range(1, EPOCH + 1):
    training(m, optimizer, training_data)
    res = testing(m, testing_data)
    print('Epoch {}/{}, Loss: {:.5f}, Accuracy: {:.2f}%'.format(
            epoch,
            EPOCH,
            res['avg_loss'],
            res['accuracy'],
        ), end='\r')

#print('\n==== Inertactive Test ====')
#print('[INFO] Enter a digit smaller than {}. ("q" to quit)'.format(2**DIGITS))
#interactive_test(m)

==== 開始訓練 ====


## 即時測試

In [37]:
def interactive_test(model):
    while True:
        num = input()
        if num == 'q':
            print('Bye~')
            return
        if int(num) >= 2**DIGITS:
            print('請輸入一個小於 {} 的數字'.format(2**DIGITS))
            continue

        ans = fizz_buzz(int(num))
        x = Variable(torch.FloatTensor([encoder(int(num))]))

        predict = model(x).data.max(1, keepdim=True)[1]
        print('預測為: {}, 答案為: {}'.format(CLASSES[predict[0][0]], CLASSES[ans]))

In [38]:
print('[提示] 請輸入一個小於 {} 的數字. (或輸入 "q" 離開)'.format(2**DIGITS))
interactive_test(m)

[INFO] 輸入一個小於 1024 的數字. (或輸入 "q" 離開)
258
Predict: Fizz, Real_Answer: Fizz
33
Predict: Fizz, Real_Answer: Fizz
35
Predict: Buzz, Real_Answer: Buzz
q
Bye~
